#  Fine-Tuning BERT for Sentiment Analysis on IMDb Dataset

##  Project Overview

This project fine-tunes the `bert-base-uncased` model—a transformer-based architecture—on the IMDb movie reviews dataset for **binary sentiment classification** (positive or negative). It utilizes the **Hugging Face Transformers** library and **PyTorch**, and is designed to run efficiently on **Google Colab** with GPU support.

---

##  Theoretical Background

### 1. BERT (Bidirectional Encoder Representations from Transformers)

####  What is BERT?
BERT, developed by Google, is a **transformer-based model** for natural language processing (NLP). Unlike traditional models, BERT processes text **bidirectionally**, meaning it learns context from both the **left and right** of a token during pre-training.

####  Architecture
- Model: `bert-base-uncased`
- Layers: 12 Transformer blocks
- Hidden Size: 768
- Attention Heads: 12
- Parameters: ~110 million

It uses **self-attention** to weigh the importance of different words in a sentence, capturing rich contextual information.

####  Pre-training Tasks
- **Masked Language Modeling (MLM):** Randomly masks 15% of the input tokens and predicts them based on surrounding context.
- **Next Sentence Prediction (NSP):** Predicts whether two input sentences follow each other in the original text.

These tasks are trained on large corpora like **Wikipedia** and **BookCorpus**.

####  Why Use BERT?
- Leverages bidirectional context
- Transfers powerful language understanding to downstream tasks like sentiment classification
- Requires less labeled data due to pre-training

---

### 2. Fine-Tuning

####  Definition
Fine-tuning means continuing the training of a **pre-trained model** on a smaller, task-specific dataset. In this case, we fine-tune BERT for **binary sentiment classification** on the IMDb reviews dataset.

####  Process
- A **classification head** (linear layer) is added on top of BERT.
- The [CLS] token’s representation is used as input to the classification head.
- The model is trained **end-to-end** (both BERT and the new head).

####  Hyperparameters
| Parameter       | Description                                     |
|----------------|-------------------------------------------------|
| Learning Rate   | `2e-5` - Stable updates to pre-trained weights |
| Batch Size      | `4` - Optimized for memory on Colab GPUs       |
| Epochs          | `1` - Sufficient for small datasets            |
| Weight Decay    | Regularization to prevent overfitting          |

---

### 3. Hugging Face Transformers

####  Overview
The [Transformers](https://huggingface.co/transformers) library by Hugging Face provides:
- Pre-trained models
- Tokenizers
- APIs for training and inference

It simplifies working with complex NLP models like BERT.

####  Components Used
- `BertTokenizer`: Tokenizes and encodes input text into BERT-compatible format using WordPiece tokenization.
- `BertForSequenceClassification`: BERT model with a classification layer.
- `Trainer` & `TrainingArguments`: High-level APIs for training, evaluation, and logging.

####  Advantages
- Abstracts away low-level PyTorch code
- Easy to use with GPU/TPU
- Supports model saving, loading, and deployment

---





In [1]:
!pip install transformers==4.44.2 datasets==3.0.1 torch==2.4.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.1/797.1 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 67.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 54.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [1]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
import torch

# Load the IMDb dataset
dataset = load_dataset("imdb")

# Load tokenizer and model
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

# Tokenize the dataset with minimal sequence length
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=256)  # Reduced from 512

# Tokenize in batches
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Use smaller dataset subsets
train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(500))  # Reduced from 1000
eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(200))    # Reduced from 200

# Minimal training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="no",  # Disable evaluation to save time/memory
    learning_rate=2e-5,       # Standard learning rate
    per_device_train_batch_size=4,  # Reduced from 8 to lower memory usage
    num_train_epochs=1,       # Already minimal
    weight_decay=0.01,        # Reduced from 0.1 for simplicity
    logging_steps=50,         # Increased from 10 to reduce logging overhead
    no_cuda=False,            # Use GPU if available (Colab default)
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
)

# Start fine-tuning
trainer.train()

# Save the fine-tuned model
model.save_pretrained("./finetuned_bert")
tokenizer.save_pretrained("./finetuned_bert")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: bluedatagpt (bluedatagpt-ghghh) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
50,0.692300


('./finetuned_bert/tokenizer_config.json',
 './finetuned_bert/special_tokens_map.json',
 './finetuned_bert/vocab.txt',
 './finetuned_bert/added_tokens.json')

In [6]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch

# Load the fine-tuned model and tokenizer
model = BertForSequenceClassification.from_pretrained("./finetuned_bert")
tokenizer = BertTokenizer.from_pretrained("./finetuned_bert")

# Example text
text = "This movie is good"

# Tokenize and predict
inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
outputs = model(**inputs)
predictions = torch.argmax(outputs.logits, dim=-1)

# Map prediction to label
labels = ["Negative", "Positive"]
print(f"Sentiment: {labels[predictions.item()]}")

Sentiment: Positive
